## import libraries

In [1]:
from compas_vol.primitives import VolBox, VolCylinder, VolPlane
from compas_vol.modifications import Overlay, Shell
from compas_vol.combinations import Intersection, Union, Subtraction
from compas_vol.microstructures import TPMS
from compas.geometry import Box, Frame, Point, Plane, Cylinder, Circle

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import add_cube

## create volumetric object (CSG tree)

In [3]:
shaft = VolBox(Box(Frame.worldXY(), 250, 30, 10), 1.5)
cyl_plane = Plane((125,0,0),(0,0,1))
roundcap = VolCylinder(Cylinder(Circle(cyl_plane, 15), 10))
handle = Union(shaft, roundcap)
gyroid = TPMS(tpmstype='Gyroid', wavelength=5.0)
shell = Shell(gyroid, 2.0, 0.5)
ol_plane = VolPlane(Plane((0,0,0), (1,0,0)))
thicken_tpms = Overlay(shell, ol_plane, 0.005)
taper = Overlay(handle, ol_plane, -0.01)
porous_handle = Intersection(thicken_tpms, taper)
solid_outer = VolCylinder(Cylinder(Circle(cyl_plane, 12), 13))
void_inner = VolCylinder(Cylinder(Circle(cyl_plane, 10), 20))
hole_reinforce = Union(porous_handle, solid_outer)
wrench = Subtraction(hole_reinforce, void_inner)


## workspace (dense grid)

In [8]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -145.0, 145.0
lby, uby = -18.0, 18.0
lbz, ubz = -8.0, 8.0
# resolution(s)
nx, ny, nz = 580, 72, 32
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [9]:
dm = wrench.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [10]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [11]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = add_cube(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))
p.add_points(np.array([[lbx,lby,lbz]]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.5459213…

2